In [27]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from datetime import datetime
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [28]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

['train', 'imdb.vocab', 'README', 'imdbEr.txt', 'test']

In [29]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['unsupBow.feat',
 'pos',
 'unsup',
 'labeledBow.feat',
 'urls_unsup.txt',
 'urls_neg.txt',
 'urls_pos.txt',
 'neg']

In [30]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [32]:
! ls -l {train_dir}

total 65216
-rw-r--r-- 1 7297 1000 21021197 Apr 12  2011 labeledBow.feat
drwxr-xr-x 2 7297 1000   360448 Apr 12  2011 neg
drwxr-xr-x 2 7297 1000   364544 Apr 12  2011 pos
-rw-r--r-- 1 7297 1000 41348699 Apr 12  2011 unsupBow.feat
-rw-r--r-- 1 7297 1000   612500 Apr 12  2011 urls_neg.txt
-rw-r--r-- 1 7297 1000   612500 Apr 12  2011 urls_pos.txt
-rw-r--r-- 1 7297 1000  2450000 Apr 12  2011 urls_unsup.txt


In [33]:
batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [34]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"esk (without

In [35]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [36]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)
result = embedding_layer(tf.constant([1,2,3,999]))
result.numpy()


array([[ 0.03296247, -0.00208326, -0.02921638,  0.04320986,  0.01757195],
       [ 0.01885838, -0.04416013, -0.01088543,  0.04276467, -0.03023729],
       [-0.02550545,  0.02240879,  0.02538315, -0.02085495, -0.04106567],
       [-0.02924993,  0.02220494, -0.00017668, -0.01962082, -0.03407626]],
      dtype=float32)

In [37]:
# 实际上就是吧[0,1,2]的0，1，2分别嵌入
result = embedding_layer(tf.constant([[0,1,2],[0,4,5]]))
print(result)
print(result.shape)

tf.Tensor(
[[[ 0.04917986 -0.00419382  0.03275848  0.04251799 -0.01536721]
  [ 0.03296247 -0.00208326 -0.02921638  0.04320986  0.01757195]
  [ 0.01885838 -0.04416013 -0.01088543  0.04276467 -0.03023729]]

 [[ 0.04917986 -0.00419382  0.03275848  0.04251799 -0.01536721]
  [ 0.04325587  0.0154233   0.04187329 -0.00137531 -0.0371448 ]
  [ 0.02690584  0.0323889  -0.02010685 -0.04651412 -0.0399121 ]]], shape=(2, 3, 5), dtype=float32)
(2, 3, 5)


In [38]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)


In [40]:
test_batch = next(iter(text_ds))
vectorize_layer(test_batch)

<tf.Tensor: shape=(1024, 100), dtype=int64, numpy=
array([[1344,   46,   91, ...,   41,   98,   80],
       [  45,   98,  119, ...,  312,  468,  775],
       [ 682,   33,  601, ...,    0,    0,    0],
       ...,
       [   2,  331,    8, ...,    2,   52,  158],
       [  30,   84,   10, ...,    1,   12,   16],
       [ 998, 1372,   11, ...,   16,  444, 2416]])>

In [41]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(), # 对于每一个样本，embeding输出的是一个100*16的，然后用这个globalAverage进行拍扁了
  Dense(16, activation='relu'),
  Dense(1)
])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])


Epoch 1/15
 1/20 [>.............................] - ETA: 0s - loss: 0.6931 - accuracy: 0.4961WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
20/20 [==============================] - 1s 64ms/step - loss: 0.6917 - accuracy: 0.5028 - val_loss: 0.6897 - val_accuracy: 0.4886

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_2 (TextVe (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________
